# 摄像头 demo

## 摄像头实时图像

In [14]:
import cv2
import ipywidgets as widgets
from IPython.display import display
import asyncio

button = widgets.Button(
    description='停止',
    button_style='danger', 
)
image = widgets.Image()
display(button, image)

# Jupyter 图像显示
def imshow(img):
    _, jpeg = cv2.imencode('.jpeg', img)
    image.value = jpeg.tobytes()

video = cv2.VideoCapture(0)

async def display_image():
    try:
        while True:
            ret, img = video.read()
            if not ret:
                break
            imshow(img)
            await asyncio.sleep(0.1)

    except asyncio.CancelledError:
        pass
    finally:
        video.release()
        
task = asyncio.ensure_future(display_image())

def on_btn_clicked(b):
    task.cancel()

button.on_click(on_btn_clicked)

Button(button_style='danger', description='停止', style=ButtonStyle())

Image(value=b'')

## 颜色识别

In [12]:
import cv2
import ipywidgets as widgets
import numpy as np
from IPython.display import display
import asyncio

video = cv2.VideoCapture(0)

button = widgets.Button(
    description='停止',
    button_style='danger', 
)
h_slider = widgets.IntRangeSlider(
    value=[100, 124],
    min=0,
    max=255,
    description='H:',
)
s_slider = widgets.IntRangeSlider(
    value=[43, 255],
    min=0,
    max=255,
    description='S:',
)
v_slider = widgets.IntRangeSlider(
    value=[46, 255],
    min=0,
    max=255,
    description='V:',
)
area_slider = widgets.IntRangeSlider(
    value=[2000, 30000],
    min=0,
    max=100000,
    step=100,
    layout=widgets.Layout(width='400px'),
    description='面积:',
)
dropdown = widgets.Dropdown(
    options=[('颜色提取', 0), ('锐化', 1), ('形态学', 2), ('形状识别', 3)],
    value=3,
    description='摄像头图像',
)
image = widgets.Image()
display(button, h_slider, s_slider, v_slider, area_slider, dropdown, image)

# Jupyter 图像显示
def imshow(img):
    _, jpeg = cv2.imencode('.jpeg', img)
    image.value = jpeg.tobytes()

async def detect_color():
    try:
        while True:
            ret, img = video.read()
            if not ret:
                break

            lower_hsv = np.array([h_slider.value[0], s_slider.value[0], v_slider.value[0]])
            upper_hsv = np.array([h_slider.value[1], s_slider.value[1], v_slider.value[1]])
                
            # 颜色遮罩
            hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
            mask = cv2.inRange(hsv, lower_hsv, upper_hsv)

            # 锐化
            sharpen_kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
            sharpen = cv2.filter2D(mask, -1, sharpen_kernel)

            # 形态学变换
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
            close = cv2.morphologyEx(sharpen, cv2.MORPH_OPEN, kernel, iterations=2)

            # 提取边缘
            cnts = cv2.findContours(close, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cnts = cnts[0] if len(cnts) == 2 else cnts[1]

            min_area = area_slider.value[0]
            max_area = area_slider.value[1]
            for c in cnts:
                area = cv2.contourArea(c)
                # 按大小筛选
                if area > min_area and area < max_area:
                    x,y,w,h = cv2.boundingRect(c)
                    cv2.putText(img, "center: ({}, {})".format(x+w/2, y+h/2), (x, y-20), cv2.FONT_HERSHEY_COMPLEX, 1, (218, 218, 3), 1)
                    cv2.rectangle(img, (x, y), (x + w, y + h), (218,218,3), 2)
            
            if dropdown.value == 0:
                imshow(mask)
            elif dropdown.value == 1:
                imshow(sharpen)
            elif dropdown.value == 2:
                imshow(close)
            else:
                imshow(img)
            await asyncio.sleep(0.1)

    except asyncio.CancelledError:
        pass
    finally:
        video.release()
        
task = asyncio.ensure_future(detect_color())

def on_btn_clicked(b):
    task.cancel()

button.on_click(on_btn_clicked)

Button(button_style='danger', description='停止', style=ButtonStyle())

IntRangeSlider(value=(100, 124), description='H:', max=255)

IntRangeSlider(value=(43, 255), description='S:', max=255)

IntRangeSlider(value=(46, 255), description='V:', max=255)

IntRangeSlider(value=(2000, 30000), description='面积:', layout=Layout(width='400px'), max=100000, step=100)

Dropdown(description='摄像头图像', index=3, options=(('颜色提取', 0), ('锐化', 1), ('形态学', 2), ('形状识别', 3)), value=3)

Image(value=b'')

## 形状识别

In [13]:
import cv2
import ipywidgets as widgets
import numpy as np
from IPython.display import display
import asyncio

video = cv2.VideoCapture(0)

button = widgets.Button(
    description='停止',
    button_style='danger', 
)
slider = widgets.IntRangeSlider(
    value=[127, 255],
    min=0,
    max=255,
    description='二值化范围:',
)
area_slider = widgets.IntRangeSlider(
    value=[8000, 17000],
    min=0,
    max=100000,
    step=100,
    layout=widgets.Layout(width='400px'),
    description='面积:',
)
dropdown = widgets.Dropdown(
    options=[('二值化', 0), ('锐化', 1), ('形态学', 2), ('形状识别', 3)],
    value=3,
    description='摄像头图像',
)
image = widgets.Image()
display(button, slider, area_slider, dropdown, image)

# Jupyter 图像显示
def imshow(img):
    _, jpeg = cv2.imencode('.jpeg', img)
    image.value = jpeg.tobytes()

async def detect_shape():
    try:
        while True:
            ret, img = video.read()
            if not ret:
                break
            # 二值化
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            _, mask = cv2.threshold(gray, slider.value[0], slider.value[1], cv2.THRESH_BINARY)

            # 锐化
            sharpen_kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
            sharpen = cv2.filter2D(mask, -1, sharpen_kernel)

            # 形态学变换
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
            close = cv2.morphologyEx(sharpen, cv2.MORPH_OPEN, kernel, iterations=2)

            # 提取边缘
            cnts = cv2.findContours(close, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cnts = cnts[0] if len(cnts) == 2 else cnts[1]

            min_area = area_slider.value[0]
            max_area = area_slider.value[1]
            for c in cnts:
                area = cv2.contourArea(c)
                # 按大小筛选
                if area > min_area and area < max_area:
                    epsilon = 0.012 * cv2.arcLength(c, True)
                    approx = cv2.approxPolyDP(c, epsilon, True)

                    # 分析几何形状
                    corners = len(approx)
                    x,y,w,h = cv2.boundingRect(c)
                    if corners >= 7:
                        cv2.putText(img, "shape: circle", (x, y-30), cv2.FONT_HERSHEY_COMPLEX, 1, (218,218,3), 1)
                    else:
                        cv2.putText(img, "sides: {}".format(corners), (x, y-30), cv2.FONT_HERSHEY_COMPLEX, 1, (218,218,3), 1)
                    cv2.putText(img, "center: ({}, {})".format(x+w/2, y+h/2), (x, y-60), cv2.FONT_HERSHEY_COMPLEX, 1, (218,218,3), 1)
                    cv2.drawContours(img, c, -1, (218,218,3),2)

            if dropdown.value == 0:
                imshow(mask)
            elif dropdown.value == 1:
                imshow(sharpen)
            elif dropdown.value == 2:
                imshow(close)
            else:
                imshow(img)
            await asyncio.sleep(0.1)

    except asyncio.CancelledError:
        pass
    finally:
        video.release()

task = asyncio.ensure_future(detect_shape())

def on_btn_clicked(b):
    task.cancel()

button.on_click(on_btn_clicked)

Button(button_style='danger', description='停止', style=ButtonStyle())

IntRangeSlider(value=(127, 255), description='二值化范围:', max=255)

IntRangeSlider(value=(8000, 17000), description='面积:', layout=Layout(width='400px'), max=100000, step=100)

Dropdown(description='摄像头图像', index=3, options=(('二值化', 0), ('锐化', 1), ('形态学', 2), ('形状识别', 3)), value=3)

Image(value=b'')